In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('/content/drive/MyDrive/frd/deceptive-opinion'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/content/drive/MyDrive/frd/deceptive-opinion/deceptive-opinion.csv


In [3]:
import re
import string
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn import datasets # import inbuild datasets

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix

In [5]:
pip install neattext

     |████████████████████████████████| 114 kB 4.3 MB/s 


In [6]:
import neattext.functions as nfx
from textblob import TextBlob

In [7]:
#df = pd.read_csv('/kaggle/input/deceptive-opinion-spam-corpus/deceptive-opinion.csv')
df = pd.read_csv('/content/drive/MyDrive/frd/amazon_reviews/new_amazon_reviews.csv')

In [8]:
#reading txt directly
#df = pd.read_csv('/content/drive/MyDrive/frd/amazon_reviews/amazon_reviews.txt', sep='\t')


# df = pd.read_csv('training.txt', sep='\t')
# sentiment_data.columns =['Class', 'Data']

In [9]:
df.head()

,DOC_ID,LABEL,RATING,VERIFIED_PURCHASE,PRODUCT_CATEGORY,PRODUCT_ID,PRODUCT_TITLE,REVIEW_TITLE,REVIEW_TEXT
0,1,__label1__,4,N,PC,B00008NG7N,"Targus PAUK10U Ultra Mini USB Keypad, Black",useful,"When least you think so, this product will sav..."
1,2,__label1__,4,Y,Wireless,B00LH0Y3NM,Note 3 Battery : Stalion Strength Replacement ...,New era for batteries,Lithium batteries are something new introduced...
2,3,__label1__,3,N,Baby,B000I5UZ1Q,"Fisher-Price Papasan Cradle Swing, Starlight",doesn't swing very well.,I purchased this swing for my baby. She is 6 m...
3,4,__label1__,4,N,Office Products,B003822IRA,Casio MS-80B Standard Function Desktop Calculator,Great computing!,I was looking for an inexpensive desk calcolat...
4,5,__label1__,4,N,Beauty,B00PWSAXAM,Shine Whitening - Zero Peroxide Teeth Whitenin...,Only use twice a week,I only use it twice a week and the results are...


In [10]:
df = df.drop(["DOC_ID", "PRODUCT_CATEGORY","PRODUCT_ID","PRODUCT_TITLE","REVIEW_TITLE"], axis=1)

In [11]:
df.head()

,LABEL,RATING,VERIFIED_PURCHASE,REVIEW_TEXT
0,__label1__,4,N,"When least you think so, this product will sav..."
1,__label1__,4,Y,Lithium batteries are something new introduced...
2,__label1__,3,N,I purchased this swing for my baby. She is 6 m...
3,__label1__,4,N,I was looking for an inexpensive desk calcolat...
4,__label1__,4,N,I only use it twice a week and the results are...


In [12]:
df = df.sample(frac=1)

In [13]:
df.head()

,LABEL,RATING,VERIFIED_PURCHASE,REVIEW_TEXT
11805,__label2__,5,Y,B+W is the brand you want if you're looking fo...
7616,__label1__,5,N,"Oh my God, really fell in love with the precio..."
4072,__label1__,2,N,"This desk is a beautiful desk, but poor qualit..."
15538,__label2__,5,Y,This particular pair of Ray-Bans are very styl...
7949,__label1__,5,N,These pedals attach very easily and stay attac...


In [14]:
from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
df['LABEL']= label_encoder.fit_transform(df['LABEL']) 

df['LABEL'].unique() 

array([1, 0])

In [15]:
df.head()

,LABEL,RATING,VERIFIED_PURCHASE,REVIEW_TEXT
11805,1,5,Y,B+W is the brand you want if you're looking fo...
7616,0,5,N,"Oh my God, really fell in love with the precio..."
4072,0,2,N,"This desk is a beautiful desk, but poor qualit..."
15538,1,5,Y,This particular pair of Ray-Bans are very styl...
7949,0,5,N,These pedals attach very easily and stay attac...


In [16]:
df.head()

,LABEL,RATING,VERIFIED_PURCHASE,REVIEW_TEXT
11805,1,5,Y,B+W is the brand you want if you're looking fo...
7616,0,5,N,"Oh my God, really fell in love with the precio..."
4072,0,2,N,"This desk is a beautiful desk, but poor qualit..."
15538,1,5,Y,This particular pair of Ray-Bans are very styl...
7949,0,5,N,These pedals attach very easily and stay attac...


In [17]:
from sklearn import preprocessing 

# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
df['VERIFIED_PURCHASE']= label_encoder.fit_transform(df['VERIFIED_PURCHASE']) 

df['VERIFIED_PURCHASE'].unique() 

array([1, 0])

In [18]:
df

,LABEL,RATING,VERIFIED_PURCHASE,REVIEW_TEXT
11805,1,5,1,B+W is the brand you want if you're looking fo...
7616,0,5,0,"Oh my God, really fell in love with the precio..."
4072,0,2,0,"This desk is a beautiful desk, but poor qualit..."
15538,1,5,1,This particular pair of Ray-Bans are very styl...
7949,0,5,0,These pedals attach very easily and stay attac...
...,...,...,...,...
1492,0,4,0,I'm not sure what everyone is all crying about...
11566,1,3,1,I have two pad trained Maltese. They use these...
17723,1,2,1,These are a regular in ear headset that are ov...
9772,0,4,0,Unlike some other products out there that leav...


In [19]:
#dataset description #truthful=1 deceptive=0
df.groupby('LABEL').describe()

RATING                                ... VERIFIED_PURCHASE                    
         count      mean       std  min  25%  ...               min  25%  50%  75%  max
LABEL                                         ...                                      
0      10500.0  4.115429  1.285663  1.0  4.0  ...               0.0  0.0  0.0  1.0  1.0
1      10500.0  4.140476  1.270898  1.0  4.0  ...               0.0  1.0  1.0  1.0  1.0

[2 rows x 16 columns]

In [20]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [21]:
# Some preprocesssing that will be common to all the text classification methods

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_char(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [22]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [23]:
df['REVIEW_TEXT'] = df['REVIEW_TEXT'].map(lambda a: clean_numbers(a))

In [24]:
df['REVIEW_TEXT'] = df['REVIEW_TEXT'].map(lambda a: clean_char(a))

In [25]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [26]:
df['REVIEW_TEXT'] = df['REVIEW_TEXT'].map(lambda a: clean_text(a))

In [27]:
df['REVIEW_TEXT']

11805    brand want look superior optic make zero sens ...
7616     god realli fell love precious littl bib lol ba...
4072     desk beauti desk poor qualiti drawer place key...
15538    particular pair ray ban stylish comfort great ...
7949     pedal attach easili stay attach like lot great...
                               ...                        
1492     sure everyon cri bib look exact like pic seem ...
11566    two pad train maltes use well grassi indoor po...
17723    regular ear headset price sound basic boarder ...
9772     unlik product leav chisel duller start save fr...
17638       one daughter came visit use love gift use time
Name: REVIEW_TEXT, Length: 21000, dtype: object

In [28]:
def get_sentiment(REVIEW_TEXT):
  blob=TextBlob(REVIEW_TEXT)
  sentiment=blob.sentiment.polarity
  if sentiment>0:
    result="Positive"
  elif sentiment<0:
    result="Negative"
  else:
    result="Neutral"
  return result

In [29]:
df['Emotion']=df['REVIEW_TEXT'].apply(get_sentiment)

In [30]:
df['Emotion']

11805    Positive
7616     Positive
4072     Negative
15538    Positive
7949     Positive
           ...   
1492     Positive
11566    Positive
17723    Positive
9772      Neutral
17638    Positive
Name: Emotion, Length: 21000, dtype: object

In [31]:
df

,LABEL,RATING,VERIFIED_PURCHASE,REVIEW_TEXT,Emotion
11805,1,5,1,brand want look superior optic make zero sens ...,Positive
7616,0,5,0,god realli fell love precious littl bib lol ba...,Positive
4072,0,2,0,desk beauti desk poor qualiti drawer place key...,Negative
15538,1,5,1,particular pair ray ban stylish comfort great ...,Positive
7949,0,5,0,pedal attach easili stay attach like lot great...,Positive
...,...,...,...,...,...
1492,0,4,0,sure everyon cri bib look exact like pic seem ...,Positive
11566,1,3,1,two pad train maltes use well grassi indoor po...,Positive
17723,1,2,1,regular ear headset price sound basic boarder ...,Positive
9772,0,4,0,unlik product leav chisel duller start save fr...,Neutral


In [32]:
df.describe()

,LABEL,RATING,VERIFIED_PURCHASE
count,21000.000000,21000.000000,21000.000000
mean,0.500000,4.127952,0.557048
std,0.500012,1.278333,0.496747
min,0.000000,1.000000,0.000000
25%,0.000000,4.000000,0.000000
50%,0.500000,5.000000,1.000000
75%,1.000000,5.000000,1.000000
max,1.000000,5.000000,1.000000


In [33]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 

# Encode labels in column 'species'. 
df['Emotion']= label_encoder.fit_transform(df['Emotion']) 

df['Emotion'].unique() 

array([2, 0, 1])

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21000 entries, 11805 to 17638
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   LABEL              21000 non-null  int64 
 1   RATING             21000 non-null  int64 
 2   VERIFIED_PURCHASE  21000 non-null  int64 
 3   REVIEW_TEXT        21000 non-null  object
 4   Emotion            21000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 984.4+ KB


In [35]:
# x = df['text']
# y = df['deceptive']

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

In [37]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(df[['Emotion']], df['LABEL'], random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(df[['Emotion','RATING']], df['LABEL'], random_state=5)
X_train, X_test, y_train, y_test = train_test_split(df[['Emotion','RATING','VERIFIED_PURCHASE']], df['LABEL'], random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(df[['RATING','VERIFIED_PURCHASE']], df['LABEL'], random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(df[['RATING']], df['LABEL'], random_state=5)
#X_train, X_test, y_train, y_test = train_test_split(df[['VERIFIED_PURCHASE']], df['LABEL'], random_state=5)
print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 21000
Number of rows in the training set: 15750
Number of rows in the test set: 5250


In [38]:
X_train

,Emotion,RATING,VERIFIED_PURCHASE
1474,1,4,0
14980,2,4,1
13812,2,5,1
17468,2,5,1
2903,1,4,0
...,...,...,...
17798,2,5,1
7123,2,5,0
671,0,1,0
5606,1,5,0


In [39]:
y_train

1474     0
14980    1
13812    1
17468    1
2903     0
        ..
17798    1
7123     0
671      0
5606     0
5992     0
Name: LABEL, Length: 15750, dtype: int64

In [40]:
X_test

,Emotion,RATING,VERIFIED_PURCHASE
19536,2,5,1
13019,2,1,1
1175,2,4,0
18321,2,4,1
2448,2,3,0
...,...,...,...
1439,2,1,0
11948,2,4,1
19969,2,5,1
16435,2,5,1


In [41]:
y_test

19536    1
13019    1
1175     0
18321    1
2448     0
        ..
1439     0
11948    1
19969    1
16435    1
18213    1
Name: LABEL, Length: 5250, dtype: int64

In [42]:
X_train, X_test, y_train, y_test

(       Emotion  RATING  VERIFIED_PURCHASE
 1474         1       4                  0
 14980        2       4                  1
 13812        2       5                  1
 17468        2       5                  1
 2903         1       4                  0
 ...        ...     ...                ...
 17798        2       5                  1
 7123         2       5                  0
 671          0       1                  0
 5606         1       5                  0
 5992         2       5                  0
 
 [15750 rows x 3 columns],        Emotion  RATING  VERIFIED_PURCHASE
 19536        2       5                  1
 13019        2       1                  1
 1175         2       4                  0
 18321        2       4                  1
 2448         2       3                  0
 ...        ...     ...                ...
 1439         2       1                  0
 11948        2       4                  1
 19969        2       5                  1
 16435        2       5   

In [43]:
# from sklearn.feature_extraction.text import CountVectorizer
# count_vector = CountVectorizer()
# print(count_vector)

In [44]:
# training_data = count_vector.fit_transform(X_train)
# testing_data = count_vector.transform(X_test)

In [45]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

MultinomialNB()

In [46]:
predictions = naive_bayes.predict(X_test)

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
mnbaccuracy = accuracy_score(y_test, predictions)
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.7826666666666666
Precision score:  0.7581252138214164
Recall score:  0.8362264150943396
F1 score:  0.7952628745738382


In [48]:
from sklearn.svm import SVC 
svc = SVC()
svc.fit(X_train, y_train)

SVC()

In [49]:
svc_predictions = svc.predict(X_test)

In [50]:
from sklearn.metrics import accuracy_score
svcaccuracy = accuracy_score(y_test,svc_predictions)
print('Accuracy score: ', format(accuracy_score(y_test,svc_predictions)))
print('Precision score: ', format(precision_score(y_test,svc_predictions)))
print('Recall score: ', format(recall_score(y_test, svc_predictions)))
print('F1 score: ', format(f1_score(y_test, svc_predictions)))

Accuracy score:  0.7826666666666666
Precision score:  0.7581252138214164
Recall score:  0.8362264150943396
F1 score:  0.7952628745738382


In [51]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [52]:
knn_predictions = knn.predict(X_test)

In [53]:
from sklearn.metrics import accuracy_score
knnaccuracy = accuracy_score(knn_predictions,y_test )
print('Accuracy score: ', format(accuracy_score(knn_predictions,y_test )))
print('Precision score: ', format(precision_score(y_test,knn_predictions)))
print('Recall score: ', format(recall_score(y_test, knn_predictions)))
print('F1 score: ', format(f1_score(y_test, knn_predictions)))

Accuracy score:  0.7742857142857142
Precision score:  0.7585598305683021
Recall score:  0.8109433962264151
F1 score:  0.7838774393580157


In [54]:
# from sklearn.naive_bayes import GaussianNB 
# gnb = GaussianNB()
# training_data1 = training_data.toarray()
# gnb.fit(training_data1, y_train)

In [55]:
# testing_data1= testing_data.toarray()
# gnb_predictions = gnb.predict(testing_data1)

In [56]:
# from sklearn.metrics import accuracy_score
# gnb_accuracy = accuracy_score(gnb_predictions,y_test )
# print('Accuracy score: ', format(accuracy_score(gnb_predictions,y_test )))
# print('Precision score: ', format(precision_score(y_test,gnb_predictions)))
# print('Recall score: ', format(recall_score(y_test, gnb_predictions)))
# print('F1 score: ', format(f1_score(y_test, gnb_predictions)))

In [57]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 2)
dtree_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=2)

In [58]:
dtree_predictions = dtree_model.predict(X_test) 

In [59]:
from sklearn.metrics import accuracy_score
dtree_accuracy = accuracy_score(dtree_predictions,y_test )
print('Accuracy score: ', format(accuracy_score(dtree_predictions,y_test )))
print('Precision score: ', format(precision_score(y_test,dtree_predictions)))
print('Recall score: ', format(recall_score(y_test, dtree_predictions)))
print('F1 score: ', format(f1_score(y_test, dtree_predictions)))

Accuracy score:  0.7826666666666666
Precision score:  0.7581252138214164
Recall score:  0.8362264150943396
F1 score:  0.7952628745738382


In [60]:
from sklearn.linear_model import SGDClassifier

In [61]:
sgd_clf = SGDClassifier()
sgd_clf.fit(X_train, y_train)

SGDClassifier()

In [62]:
sgdpredicted = sgd_clf.predict(X_test)

In [63]:
from sklearn.metrics import accuracy_score
sgd_accuracy = accuracy_score(sgdpredicted,y_test )
print('Accuracy score: ', format(accuracy_score(sgdpredicted,y_test )))
print('Precision score: ', format(precision_score(y_test,sgdpredicted)))
print('Recall score: ', format(recall_score(y_test, sgdpredicted)))
print('F1 score: ', format(f1_score(y_test, sgdpredicted)))

Accuracy score:  0.7826666666666666
Precision score:  0.7581252138214164
Recall score:  0.8362264150943396
F1 score:  0.7952628745738382


In [64]:
from sklearn.linear_model import LogisticRegression

In [65]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [66]:
lrpredicted = lr.predict(X_test)

In [67]:
from sklearn.metrics import accuracy_score
lr_accuracy = accuracy_score(lrpredicted,y_test )
print('Accuracy score: ', format(accuracy_score(lrpredicted,y_test )))
print('Precision score: ', format(precision_score(y_test,lrpredicted)))
print('Recall score: ', format(recall_score(y_test, lrpredicted)))
print('F1 score: ', format(f1_score(y_test, lrpredicted)))

Accuracy score:  0.7826666666666666
Precision score:  0.7581252138214164
Recall score:  0.8362264150943396
F1 score:  0.7952628745738382


In [68]:
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier(n_estimators=50, base_estimator= None,learning_rate=0.03, random_state = 1)
adaboost.fit(X_train,y_train)

AdaBoostClassifier(learning_rate=0.03, random_state=1)

In [69]:
adaboostpredicted = adaboost.predict(X_test)

In [70]:
from sklearn.metrics import accuracy_score
adaboost_accuracy = accuracy_score(adaboostpredicted,y_test )
print('Accuracy score: ', format(accuracy_score(adaboostpredicted,y_test )))
print('Precision score: ', format(precision_score(y_test,adaboostpredicted)))
print('Recall score: ', format(recall_score(y_test, adaboostpredicted)))
print('F1 score: ', format(f1_score(y_test, adaboostpredicted)))

Accuracy score:  0.7826666666666666
Precision score:  0.7581252138214164
Recall score:  0.8362264150943396
F1 score:  0.7952628745738382


In [71]:
rf = RandomForestClassifier(n_estimators=100)

In [72]:
bag_clf = BaggingClassifier(base_estimator=rf, n_estimators=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=100,
                  n_jobs=-1, random_state=42)

In [73]:
#bag_clf.score(X_train,y_train),bag_clf.score(X_test,y_test)
bag_clfpredicted = bag_clf.predict(X_test)

In [74]:
from sklearn.metrics import accuracy_score
bag_clfpredicted_accuracy = accuracy_score(bag_clfpredicted,y_test )
print('Accuracy score: ', format(accuracy_score(bag_clfpredicted,y_test )))
print('Precision score: ', format(precision_score(y_test,bag_clfpredicted)))
print('Recall score: ', format(recall_score(y_test, bag_clfpredicted)))
print('F1 score: ', format(f1_score(y_test, bag_clfpredicted)))

Accuracy score:  0.7826666666666666
Precision score:  0.7581252138214164
Recall score:  0.8362264150943396
F1 score:  0.7952628745738382


In [75]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

In [76]:
print('Multinomial Naive Bayes:',mnbaccuracy)
#print('Gausian Naive Bayes:',gnb_accuracy)
print('Decision tree:',dtree_accuracy)
print('Support Vector Classifier:',svcaccuracy)
print('K-Nearest Neighbour:',knnaccuracy)
print('Stochastic Gradient Descent:',sgd_accuracy)
print('LogisticRegression:',lr_accuracy)
print('AdaBoostClassifier:',adaboost_accuracy)
print('BaggingClassifier',bag_clfpredicted_accuracy)

Multinomial Naive Bayes: 0.7826666666666666
Decision tree: 0.7826666666666666
Support Vector Classifier: 0.7826666666666666
K-Nearest Neighbour: 0.7742857142857142
Stochastic Gradient Descent: 0.7826666666666666
LogisticRegression: 0.7826666666666666
AdaBoostClassifier: 0.7826666666666666
BaggingClassifier 0.7826666666666666


In [77]:
#Emotion','RATING','VERIFIED_PURCHASE
# Multinomial Naive Bayes: 0.7897142857142857
# Decision tree: 0.7897142857142857
# Support Vector Classifier: 0.7897142857142857
# K-Nearest Neighbour: 0.6506666666666666
# Stochastic Gradient Descent: 0.7897142857142857
# LogisticRegression: 0.7897142857142857

In [78]:
#'RATING','VERIFIED_PURCHASE'
# Multinomial Naive Bayes: 0.7843809523809524
# Decision tree: 0.7843809523809524
# Support Vector Classifier: 0.7843809523809524
# K-Nearest Neighbour: 0.7843809523809524
# Stochastic Gradient Descent: 0.7843809523809524
# LogisticRegression: 0.7843809523809524

In [79]:
# VERIFIED_PURCHASE
# Multinomial Naive Bayes: 0.49714285714285716
# Decision tree: 0.7876190476190477
# Support Vector Classifier: 0.7876190476190477
# K-Nearest Neighbour: 0.49714285714285716
# Stochastic Gradient Descent: 0.7876190476190477
# LogisticRegression: 0.7876190476190477

In [80]:
#Emotion','RATING'
# Multinomial Naive Bayes: 0.4900952380952381
# Decision tree: 0.49123809523809525
# Support Vector Classifier: 0.4908571428571429
# K-Nearest Neighbour: 0.508
# Stochastic Gradient Descent: 0.5104761904761905
# LogisticRegression: 0.49028571428571427

In [81]:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=2)
# kmeans.fit(X_train)
# y_kmeans = kmeans.predict(X_train)

In [82]:
# from nltk.stem.porter import PorterStemmer
# from nltk.corpus import stopwords

In [83]:
# ps = PorterStemmer()

In [84]:
# stemmed_dataset = []
# for i in range(0,21000):
#     stemmed_array = df['REVIEW_TEXT'][i].split()
#     stemmed = [ps.stem(word) for word in stemmed_array if not word in set(stopwords.words('english'))]
#     stemmed = ' '.join(stemmed)
#     stemmed_dataset.append(stemmed)

In [85]:
# print(stemmed[0:21000])

In [86]:
# # roc curve and auc
# from sklearn.datasets import make_classification
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score
# from matplotlib import pyplot

# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(x, y, test_size=0.3, random_state=2)
# #vectorizer
# training_data = count_vector.fit_transform(trainX)
# testing_data = count_vector.transform(testX)
# # fit a model
# model = KNeighborsClassifier(n_neighbors=3)
# model.fit(training_data, trainy)
# # predict probabilities
# probs = model.predict_proba(testing_data)
# # keep probabilities for the positive outcome only
# probs = probs[:, 1]
# # calculate AUC
# auc = roc_auc_score(testy, probs)
# print('AUC: %.3f' % auc)
# # calculate roc curve
# fpr, tpr, thresholds = roc_curve(testy, probs)
# # plot no skill
# pyplot.plot([0, 1], [0, 1], linestyle='--')
# # plot the roc curve for the model
# pyplot.plot(fpr, tpr, marker='.')
# # show the plot
# pyplot.show()

In [87]:
# # roc curve and auc
# from sklearn.datasets import make_classification
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score
# from matplotlib import pyplot

# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(x, y, test_size=0.3, random_state=2)
# #vectorizer
# training_data = count_vector.fit_transform(trainX)
# testing_data = count_vector.transform(testX)
# # fit a model
# model = LogisticRegression()
# model.fit(training_data, trainy)
# # predict probabilities
# probs = model.predict_proba(testing_data)
# # keep probabilities for the positive outcome only
# probs = probs[:, 1]
# # calculate AUC
# auc = roc_auc_score(testy, probs)
# print('AUC: %.3f' % auc)
# # calculate roc curve
# fpr, tpr, thresholds = roc_curve(testy, probs)
# # plot no skill
# pyplot.plot([0, 1], [0, 1], linestyle='--')
# # plot the roc curve for the model
# pyplot.plot(fpr, tpr, marker='.')
# # show the plot
# pyplot.show()

In [88]:
# # roc curve and auc
# from sklearn.datasets import make_classification
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score
# from matplotlib import pyplot

# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(x, y, test_size=0.3, random_state=2)
# #vectorizer
# training_data = count_vector.fit_transform(trainX)
# testing_data = count_vector.transform(testX)
# # fit a model
# model = DecisionTreeClassifier(max_depth = 2)
# model.fit(training_data, trainy)
# # predict probabilities
# probs = model.predict_proba(testing_data)
# # keep probabilities for the positive outcome only
# probs = probs[:, 1]
# # calculate AUC
# auc = roc_auc_score(testy, probs)
# print('AUC: %.3f' % auc)
# # calculate roc curve
# fpr, tpr, thresholds = roc_curve(testy, probs)
# # plot no skill
# pyplot.plot([0, 1], [0, 1], linestyle='--')
# # plot the roc curve for the model
# pyplot.plot(fpr, tpr, marker='.')
# # show the plot
# pyplot.show()

In [89]:
# # roc curve and auc
# from sklearn.datasets import make_classification
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score
# from matplotlib import pyplot

# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(x, y, test_size=0.3, random_state=2)
# #vectorizer
# training_data = count_vector.fit_transform(trainX)
# testing_data = count_vector.transform(testX)
# # fit a model
# model = GaussianNB()
# training_data1 = training_data.toarray()
# model.fit(training_data1, trainy)
# # predict probabilities
# testing_data1 = testing_data.toarray()
# probs = model.predict_proba(testing_data1)
# # keep probabilities for the positive outcome only
# probs = probs[:, 1]
# # calculate AUC
# auc = roc_auc_score(testy, probs)
# print('AUC: %.3f' % auc)
# # calculate roc curve
# fpr, tpr, thresholds = roc_curve(testy, probs)
# # plot no skill
# pyplot.plot([0, 1], [0, 1], linestyle='--')
# # plot the roc curve for the model
# pyplot.plot(fpr, tpr, marker='.')
# # show the plot
# pyplot.show()

In [90]:
# # roc curve and auc
# from sklearn.datasets import make_classification
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score
# from matplotlib import pyplot

# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(x, y, test_size=0.3, random_state=2)
# #vectorizer
# training_data = count_vector.fit_transform(trainX)
# testing_data = count_vector.transform(testX)
# # fit a model
# model = MultinomialNB()
# model.fit(training_data, trainy)
# # predict probabilities
# probs = model.predict_proba(testing_data)
# # keep probabilities for the positive outcome only
# probs = probs[:, 1]
# # calculate AUC
# auc = roc_auc_score(testy, probs)
# print('AUC: %.3f' % auc)
# # calculate roc curve
# fpr, tpr, thresholds = roc_curve(testy, probs)
# # plot no skill
# pyplot.plot([0, 1], [0, 1], linestyle='--')
# # plot the roc curve for the model
# pyplot.plot(fpr, tpr, marker='.')
# # show the plot
# pyplot.show()

In [91]:
# # roc curve and auc
# from sklearn.datasets import make_classification
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_curve
# from sklearn.metrics import roc_auc_score
# from matplotlib import pyplot

# # split into train/test sets
# trainX, testX, trainy, testy = train_test_split(x, y, test_size=0.3, random_state=2)
# #vectorizer
# training_data = count_vector.fit_transform(trainX)
# testing_data = count_vector.transform(testX)
# # fit a model
# model = SVC()
# model.fit(training_data, trainy)
# # predict probabilities
# probs = model.predict_proba(testing_data)
# # keep probabilities for the positive outcome only
# probs = probs[:, 1]
# # calculate AUC
# auc = roc_auc_score(testy, probs)
# print('AUC: %.3f' % auc)
# # calculate roc curve
# fpr, tpr, thresholds = roc_curve(testy, probs)
# # plot no skill
# pyplot.plot([0, 1], [0, 1], linestyle='--')
# # plot the roc curve for the model
# pyplot.plot(fpr, tpr, marker='.')
# # show the plot
# pyplot.show()